<a href="https://colab.research.google.com/github/Theseyh/Big-Data-Framework/blob/main/BDF_02_Basic_operations_on_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#00 - Configuration of Apache Spark on Collaboratory

---




###Installing Java, Spark, and Findspark


---


This code installs Apache Spark 2.2.1, Java 8, and [Findspark](https://github.com/minrk/findspark), a library that makes it easy for Python to find Spark.

In [7]:
import os

os.environ["SPARK_VERSION"] = "spark-3.5.3"
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget  http://apache.osuosl.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!echo $SPARK_VERSION-bin-hadoop3.tgz
!rm $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 257 kB in 1s (183 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
--2024-11-25 10:37:2

### Set Environment Variables
Set the locations where Spark and Java are installed.

In [8]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark/"
os.environ["DRIVE_DATA"] = "/content/gdrive/My Drive/Big Data Framework/data/"

!rm /content/spark
!ln -s /content/$SPARK_VERSION-bin-hadoop3 /content/spark
!export PATH=$PATH:$SPARK_HOME/bin:$SPARK_HOME/sbin
!echo $SPARK_HOME
!env |grep  "DRIVE_DATA"

/content/spark/
DRIVE_DATA=/content/gdrive/My Drive/Big Data Framework/data/


### Start a SparkSession
This will start a local Spark session.

In [9]:
!python -V

import findspark
findspark.init()

from pyspark import SparkContext
sc = SparkContext.getOrCreate()

# Example: shows the PySpark version
print("PySpark version {0}".format(sc.version))

# Example: parallelise an array and show the 2 first elements
sc.parallelize([2, 3, 4, 5, 6]).cache().take(2)

Python 3.10.12
PySpark version 3.5.3


[2, 3]

In [10]:
from pyspark.sql import SparkSession
# We create a SparkSession object (or we retrieve it if it is already created)
spark = SparkSession \
.builder \
.appName("My application") \
.config("spark.some.config.option", "some-value") \
.master("local[4]") \
.getOrCreate()
# We get the SparkContext
sc = spark.sparkContext

In [11]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [12]:
!ls "$DRIVE_DATA"

2015-summary.csv       books   cite75_99.txt.tar.bz2  italianPosts.csv.bz2  people.txt	 sw.txt
apat63_99.txt.tar.bz2  by-day  country_codes.txt      myscript.py	    quijote.txt




---


# 02 - Basic operations on Spark

- Spark operates with immutable and distributed collections of elements, managing them in parallel
    - Structured API: DataFrames and DataSets
    - Low-level API: RDDs

-   Operations on these collections
    -   Creation
    -   Transformations (sorting, filtering, etc.)
    -   Actions to obtain results

-   Spark automatically distributes data and parallelises operations



## Example: creation of a DataFrame from a CSV file

**Note:** To learn how to upload a file into collaboratory from your machine or from Google Drive, check [this link](https://colab.research.google.com/notebooks/io.ipynb)


### Option 1: Uploading the *2015-summary.csv* CSV file from your computer

1.   List item
2.   List item



In [13]:
from google.colab import files
import pandas as pd
import io

uploaded = files.upload()


Saving 2015-summary.csv to 2015-summary.csv


In [14]:
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  df = pd.read_csv(io.StringIO(uploaded[fn].decode('utf-8')))
  print(format(df.head()))

User uploaded file "2015-summary.csv" with length 7080 bytes
  DEST_COUNTRY_NAME ORIGIN_COUNTRY_NAME  count
0     United States             Romania     15
1     United States             Croatia      1
2     United States             Ireland    344
3             Egypt       United States     15
4     United States               India     62


In [15]:
!ls -lh 2015-summary.csv
!head 2015-summary.csv

-rw-r--r-- 1 root root 7.0K Nov 25 10:41 2015-summary.csv
DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,15
United States,Croatia,1
United States,Ireland,344
Egypt,United States,15
United States,India,62
United States,Singapore,1
United States,Grenada,62
Costa Rica,United States,588
Senegal,United States,40


### Option 2: Uploading the CSV file from Google Drive

In [16]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [17]:
!head "$DRIVE_DATA/2015-summary.csv"

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,15
United States,Croatia,1
United States,Ireland,344
Egypt,United States,15
United States,India,62
United States,Singapore,1
United States,Grenada,62
Costa Rica,United States,588
Senegal,United States,40


### Creating the DataFrame

In this example, Spark infers the data schema automatically

  - It is better to specify the schema in a explicit way, as we will see later

We define the first line to be the header.

In [18]:
from pyspark.sql import SparkSession
# We create a SparkSession object (or we retrieve it if it is already created)
spark = SparkSession \
.builder \
.appName("My application") \
.config("spark.some.config.option", "some-value") \
.master("local[4]") \
.getOrCreate()

flightData2015 = (spark
    .read
    .option("inferSchema", "true")
    .option("header", "true")
    .csv(os.environ["DRIVE_DATA"] +"/2015-summary.csv"))

flightData2015.printSchema()

flightData2015.show(5)
print(flightData2015.count())

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: integer (nullable = true)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows

256


## Rows

Rows in a DataFrame are objects of `Row`  type

- Row API in Python: https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.Row.html#pyspark.sql.Row

### Row manipulation

In [19]:
# Get the two first rows of the DataFrame
row1_2 = flightData2015.take(2)
print(row1_2)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15), Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1)]


In [20]:
# Get the first row as a Python dictionary
print(row1_2[0].asDict())

{'DEST_COUNTRY_NAME': 'United States', 'ORIGIN_COUNTRY_NAME': 'Romania', 'count': 15}


## Partitions

The elements in a DataFrame (or DataSet or RDD) are splitted between the nodes of the cluster, dividing the collection in partitions. Each partition is then processed by a given executor.

-  The number of partitions by default is a function of the cluster size (total number of cores from every executor) and the data size (number of blocks of the files in HDFS)
-  In the case of an RDD, a different partition size can be specified at creation time.
- The partition size can be modified once they are created.

![Partitioning](https://docs.google.com/drawings/d/1GAasfY7P7uaMXhvGHuZ1nOqPqv6TrE7-N96RqUn1NqE/pub?w=960&h=540)



In [21]:
print("Number of partitions: {0}"
    .format(flightData2015.rdd.getNumPartitions()))

# Create a new DataFrame with 4 partitions
flightData2015_4P = flightData2015.repartition(4)
print("Number of partitions: {0}"
    .format(flightData2015_4P.rdd.getNumPartitions()))

Number of partitions: 1
Number of partitions: 4


##Transformations vs Actions

### Transformations

Operations that transform data

  - Origin data are not transformed ( *immutability* )
  - Transformations are computed in a "lazy" way ( *lazyness* ),  in the sense that they do not actually do anything until an action is executed.

Two types:

  - *Narrow* Transformations
    - Each input partition contributes to a single output partition
    - The number of partitions is not modified
    - Typically performed in memory
  - *Wide* Transformations
    - Each output partition depends on several (or all) input partitions
    - They imply data shuffling
    - The number of partitions can be modified
    - They may imply disk writes
    
Examples:
* map
* filter
* replace

In [22]:
# Narrow transformation example
flightData2015_EEUU = flightData2015.replace("United States", "Estados Unidos")
flightData2015_EEUU.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|   Estados Unidos|            Romania|   15|
|   Estados Unidos|            Croatia|    1|
|   Estados Unidos|            Ireland|  344|
|            Egypt|     Estados Unidos|   15|
|   Estados Unidos|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



In [23]:
# Wide transformation example
flightData2015_Ord = flightData2015_EEUU.sort("count", ascending=False)
flightData2015_Ord.cache()
flightData2015_Ord.show(5)  #we don't want to force an action

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|   Estados Unidos|     Estados Unidos|370002|
|   Estados Unidos|             Canada|  8483|
|           Canada|     Estados Unidos|  8399|
|   Estados Unidos|             Mexico|  7187|
|           Mexico|     Estados Unidos|  7140|
+-----------------+-------------------+------+
only showing top 5 rows



### Actions

They return a result to the driver program, forcing therefore to perform the pending transformations

  - When an action is triggered, a *plan* is created with the transformations needed to obtain the requested data
    - A Directed Acyclic Graph (DAG) is created to connect the transformations to apply
    - Spark will optimise this graph, removing unnecessary tranformations and joining them when possible
  - Actions translate the DAG into an execution plan

Types of actions:

  - Actions to show data in the console
  - Actions to convert Spark data into language-related data
  - Actions to write data to disk
  
Examples:
* reduce
* collect
* take
* show


In [24]:
# Action example
print("Number of rows in the table: {0}".format(flightData2015_Ord.count()))

print(flightData2015_Ord.take(3))

flightData2015_Ord.show()

Number of rows in the table: 256
[Row(DEST_COUNTRY_NAME='Estados Unidos', ORIGIN_COUNTRY_NAME='Estados Unidos', count=370002), Row(DEST_COUNTRY_NAME='Estados Unidos', ORIGIN_COUNTRY_NAME='Canada', count=8483), Row(DEST_COUNTRY_NAME='Canada', ORIGIN_COUNTRY_NAME='Estados Unidos', count=8399)]
+------------------+-------------------+------+
| DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+------------------+-------------------+------+
|    Estados Unidos|     Estados Unidos|370002|
|    Estados Unidos|             Canada|  8483|
|            Canada|     Estados Unidos|  8399|
|    Estados Unidos|             Mexico|  7187|
|            Mexico|     Estados Unidos|  7140|
|    United Kingdom|     Estados Unidos|  2025|
|    Estados Unidos|     United Kingdom|  1970|
|             Japan|     Estados Unidos|  1548|
|    Estados Unidos|              Japan|  1496|
|           Germany|     Estados Unidos|  1468|
|    Estados Unidos| Dominican Republic|  1420|
|Dominican Republic|     Estados Un

### DAG example
Each job is represented by a graph (specifically a directed acyclic graph (DAG)):

![DAG](http://2.bp.blogspot.com/-5sDP78mSdlw/Ur3szYz1HpI/AAAAAAAABCo/Aak2Xn7TmnI/s1600/p2.png)